In [1]:
import sys
import numpy as np
import cv2

In [2]:
im = cv2.imread('C:/Users/dell/Desktop/train.png')
cv2.imshow('base img',im)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
cv2.imshow('grayscale img',gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
blur = cv2.GaussianBlur(gray,(5,5),0)
cv2.imshow('blurred img',blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
thresh = cv2.adaptiveThreshold(blur,255,1,1,11,2)
cv2.imshow('threshed img',thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# Detecting The Contours
im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

samples =  np.empty((0,100))
responses = []
keys = [i for i in range(48,58)]

for cnt in contours:
    if cv2.contourArea(cnt)>50:
        [x,y,w,h] = cv2.boundingRect(cnt)

        if  h>28:
            cv2.rectangle(im,(x,y),(x+w,y+h),(0,0,255),2)
            roi = thresh[y:y+h,x:x+w]
            roismall = cv2.resize(roi,(10,10))
            cv2.imshow('norm',im)
            key = cv2.waitKey(0)

            if key == 27:  # (escape to quit)
                sys.exit()
            elif key in keys:
                responses.append(int(chr(key)))
                sample = roismall.reshape((1,100))
                samples = np.append(samples,sample,0)

responses = np.array(responses,np.float32)
responses = responses.reshape((responses.size,1))
print("training complete")

np.savetxt('generalsamples.data',samples)
np.savetxt('generalresponses.data',responses)

training complete


In [11]:
# Training Data
samples = np.loadtxt('generalsamples.data',np.float32)
responses = np.loadtxt('generalresponses.data',np.float32)
responses = responses.reshape((responses.size,1))

model = cv2.ml.KNearest_create()
model.train(samples,cv2.ml.ROW_SAMPLE,responses)

True

In [14]:
im3 = cv2.imread('C:/Users/dell/Desktop/test.png')
out = np.zeros(im3.shape,np.uint8)
gray = cv2.cvtColor(im3,cv2.COLOR_BGR2GRAY)
thresh = cv2.adaptiveThreshold(gray,255,1,1,11,2)

im4,contours,hierarchy = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours:
    if cv2.contourArea(cnt)>50:
        [x,y,w,h] = cv2.boundingRect(cnt)
        if  h>28:
            cv2.rectangle(im3,(x,y),(x+w,y+h),(0,255,0),2)
            roi = thresh[y:y+h,x:x+w]
            roismall = cv2.resize(roi,(10,10))
            roismall = roismall.reshape((1,100))
            roismall = np.float32(roismall)
            retval, results, neigh_resp, dists = model.findNearest(roismall, 1)
            string = str(int((results[0][0])))
            cv2.putText(out,string,(x,y+h),0,1,(0,255,0))

cv2.imshow('Test',im3)
cv2.imshow('out',out)
cv2.waitKey(0)

-1

In [21]:
im3 = cv2.imread('C:/Users/dell/Desktop/fig.jpg')
out = np.zeros(im3.shape,np.uint8)
gray = cv2.cvtColor(im3,cv2.COLOR_BGR2GRAY)
thresh = cv2.adaptiveThreshold(gray,255,1,1,11,2)

im4,contours,hierarchy = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours:
    if cv2.contourArea(cnt)>50:
        [x,y,w,h] = cv2.boundingRect(cnt)
        if  h>28:
            cv2.rectangle(im3,(x,y),(x+w,y+h),(0,255,0),2)
            roi = thresh[y:y+h,x:x+w]
            roismall = cv2.resize(roi,(10,10))
            roismall = roismall.reshape((1,100))
            roismall = np.float32(roismall)
            retval, results, neigh_resp, dists = model.findNearest(roismall, 1)
            string = str(int((results[0][0])))
            cv2.putText(out,string,(x,y+h),0,1,(0,255,0))

cv2.imshow('Test',im3)
cv2.imshow('out',out)
cv2.waitKey(0)

-1